In [15]:
# Useful libs
from sklearn.datasets import fetch_20newsgroups
import pickle
import pandas as pd
import numpy as np
import csv
import re
# STTM lib from Github
from gsdmm import MovieGroupProcess

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# Custom python scripts for preprocessing, prediction and
# visualization that I will define more in depth later
#from preprocessing import tokenize
#from topic_allocation import top_words, topic_attribution
#from visualisation import plot_topic_notebook
# Load the 20NewsGroups dataset from sklearn
#cats = ['talk.politics.mideast', 'comp.windows.x', 'sci.space']
#newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)

In [4]:
columns = ['date', 'hashtags', 'name', 'likes_count', 'replies_count', 'retweets_count', 'tweet','username']

In [58]:
who_df = pd.read_csv('nytimes_covid.csv',encoding='utf-8')
who_df.drop_duplicates(keep="first", inplace=True)
who_df = who_df[columns]
who_df = who_df.set_index('date')
who_df.index = pd.to_datetime(who_df.index, format="%Y-%m-%d")

print (who_df.tweet[1])

An earlier version of this article misstated the date of the U.S.’s first confirmed case of coronavirus through travel. It was Jan. 20, not Jan. 8. We've deleted a tweet that also included the error.  https://nyti.ms/39lcxWs 


In [59]:

def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

#my_stopwords = nltk.corpus.stopwords.words('english')
#stopwords_perso=["said"]
my_stopwords=set(stopwords.words('english'))
#my_stopwords.append(stopwords_perso)
stemmer= PorterStemmer()

word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(tweet, bigrams=False):
    
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    
    tweet_token_list = word_tokenize(tweet)
    tweet_token_list= [stemmer.stem(word) for word in tweet_token_list if word not in my_stopwords]
    
#    tweet_token_list = [word for word in tweet.split(' ')
#                            if word not in my_stopwords] # remove stopwords

    
#    tweet_token_list = [word_rooter(word) if '#' not in word else word
#                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
        
   
    #tweet = ' '.join(tweet_token_list)
    for x in tweet_token_list:
        x.replace("u"," ")
        
    return tweet_token_list



who_df.tweet=who_df.tweet.apply(remove_links)
who_df.tweet=who_df.tweet.apply(remove_users)
who_df.tweet=who_df.tweet.apply(clean_tweet)


who_df.tweet[1]


AttributeError: 'set' object has no attribute 'append'

In [53]:

i=0
# Input format for the model : list of strings (list of tokens)
tweets=who_df["tweet"].tolist()
print len(tweets)
for tweet in tweets:
    for x in tweet:
        if x=='u' or (x==u'\u2019') or (x==u'\u201c') or (x==u'\u2014') or (x==u'\u201d') : #remove some special characters and the years
            tweet.remove(x)
            i+=1
print i   
vocab = set(x for tweet in tweets for x in tweet)
#vocab = set(x for tweet in tweets for x in tweet if u'\x99' not in x)
#vocab = [x.replace(u'\x99', '') for x in vocab]
#vocab = [x.replace(u'\x98', '') for x in vocab]
n_terms = len(vocab)

print("Voc size:", n_terms)
print("Number of documents:", len(tweets))


10848
2991
('Voc size:', 13688)
('Number of documents:', 10848)


In [54]:
# Train a new model 

# Init of the Gibbs Sampling Dirichlet Mixture Model algorithm
mgp = MovieGroupProcess(K=10, alpha=0.1, beta=0.1, n_iters=5)

#vocab = set(x for tweet in tweets for x in tweet)
n_terms = len(vocab)
n_tweets = len(tweets)

# Fit the model on the data given the chosen seeds
y = mgp.fit(tweets, n_terms)

# Save model
with open('model_v2.model', "wb") as f:
    pickle.dump(mgp, f)
    f.close()

In stage 0: transferred 8598 clusters with 10 clusters populated
In stage 1: transferred 4833 clusters with 10 clusters populated
In stage 2: transferred 2845 clusters with 10 clusters populated
In stage 3: transferred 1922 clusters with 10 clusters populated
In stage 4: transferred 1415 clusters with 10 clusters populated


In [55]:
tweet_count = np.array(mgp.cluster_doc_count)
print('Number of tweets per topic :', tweet_count)
print('*'*20)
# Topics sorted by the number of document they are allocated to
top_index = tweet_count.argsort()[-10:][::-1]
print('Most important clusters (by number of tweets inside):', top_index)
print('*'*20)
# Show the top 5 words in term frequency for each cluster 
#top_words(mgp.cluster_word_distribution, top_index, 5)

('Number of tweets per topic :', array([1245, 1142,  641, 1193, 1248, 1144, 1661,  682, 1355,  537]))
********************
('Most important clusters (by number of tweets inside):', array([6, 8, 4, 0, 3, 5, 1, 7, 2, 9]))
********************


In [56]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print('Cluster %s : %s'%(cluster,sort_dicts))
        print(' — — — — — — — — — ')

In [57]:
top_words(mgp.cluster_word_distribution, top_index, 5)

Cluster 6 : [(u'\u201d', 471), (u'coronaviru', 362), (u'new', 260), (u'write', 254), (u'one', 223)]
 — — — — — — — — — 
Cluster 8 : [(u'democrat', 498), (u'presidenti', 306), (u'day', 303), (u'sander', 255), (u'need', 249)]
 — — — — — — — — — 
Cluster 4 : [(u'presid', 755), (u'trump', 721), (u'impeach', 443), (u'senat', 271), (u'hous', 263)]
 — — — — — — — — — 
Cluster 0 : [(u'\u201d', 436), (u'new', 186), (u'write', 164), (u'year', 161), (u'critic', 136)]
 — — — — — — — — — 
Cluster 3 : [(u'coronaviru', 826), (u'new', 347), (u'outbreak', 250), (u'case', 238), (u'china', 235)]
 — — — — — — — — — 
Cluster 5 : [(u'said', 305), (u'year', 235), (u'new', 233), (u'peopl', 222), (u'kill', 207)]
 — — — — — — — — — 
Cluster 1 : [(u'coronaviru', 247), (u'trump', 176), (u'presid', 164), (u'said', 153), (u'chines', 149)]
 — — — — — — — — — 
Cluster 7 : [(u'news', 99), (u'write', 95), (u'presid', 90), (u'coronaviru', 89), (u'\u201d', 88)]
 — — — — — — — — — 
Cluster 2 : [(u'\u201d', 142), (u'said',